In [59]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [60]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [61]:
clients = utils.set_data(True)

Creating node1 on 0.0.0.0:11111.
Creating node2 on 0.0.0.0:11111.
Creating node3 on 0.0.0.0:11111.
Creating node4 on 0.0.0.0:11111.
Creating node5 on 0.0.0.0:11111.


In [62]:
selected_model = Supported_modles.logistic_regression

In [63]:
f1_before = []
for client in clients:
    client.split_data()
    # client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    # client.prep_data()
    client.train_model(selected_model)
    f1_before.append(client.f1)

In [67]:
f1_before

[0.8009012956124428,
 0.9790305276504437,
 0.8685616584740329,
 0.5609756097560975,
 0.9431055900621117]

In [68]:
selected_model = Supported_modles.SGD_classifier

In [69]:
# clients = utils.set_data(True)
for client in clients:
    # client.split_data()
    client.init_empty_model(selected_model,None)

In [70]:
fedavg = Fedavg("global", 0.1)
fedavg.init_global_model(selected_model, None,78)

In [71]:
number_of_rounds = 20
batch_size = 0.1
epochs = 20
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!


In [72]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.4880437613961969, 0.3428811586706324, 0.35639344262295086, 0.08504252126063032, 0.7217021276595745]


In [73]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference -0.31285753421624596
Difference -0.6361493689798113
Difference -0.5121682158510821
Difference -0.47593308849546717
Difference -0.22140346240253717
